In [ ]:
pip install moviepy

In [ ]:
import torch
import torchaudio
import os
import pandas as pd
from transformers import AutoFeatureExtractor, WhisperModel

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

In [ ]:
# Set the paths for input audio files and output feature file
audio_directory = "/mnt/c/users/admin/desktop/F21RP/datasets/ravdess/audios"
output_file = "/mnt/c/users/admin/desktop/git hub/ml_projects/features/audio/features.csv"

In [ ]:
# Initialize the WhisperModel and AutoFeatureExtractor
model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

In [ ]:
# Define a function to load the audio file
def load_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    return waveform

In [ ]:
# Define a function to extract features from audio and save them
def extract_features_and_save(audio_directory, output_file):
    features_list = []
    
    # Traverse through the audio directory and subdirectories
    for root, dirs, files in os.walk(audio_directory):
        for audio_file in files:
            audio_path = os.path.join(root, audio_file)
            
            # Load the audio file
            audio = load_audio(audio_path)
            
            # Extract features using the feature extractor
            inputs = feature_extractor(audio, return_tensors="pt", padding=True, truncation=True)
            input_features = inputs["input_features"]
            padding_shape = (model.config.max_position_embeddings - input_features.shape[1])
            padded_features = torch.nn.functional.pad(input_features, (0, paddiing_shape))
            
            #pass the padded features through the whisper model
            decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id
            last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state
            
            # Append the features and metadata to the list
            features_list.append({
                'path': audio_path,
                'features': last_hidden_state.squeeze()  # Remove unnecessary dimensions
            })
    
    # Create a DataFrame from the features list
    features_df = pd.DataFrame(features_list)
    
    # Save the DataFrame to a CSV file
    features_df.to_csv(output_file, index=False)
    
    print("Audio features saved successfully!")

# Call the function to extract features and save them
extract_features_and_save(audio_directory, output_file)

In [ ]:
import os
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

# Set the path to the root folder containing the audio files
root_folder = "/mnt/c/users/admin/desktop/F21RP/datasets/ravdess/audios"

# Set the path to the folder where you want to save the extracted features
features_folder = "/mnt/c/users/admin/desktop/git hub/ml_projects/features/audio/"

# Load the Whisper model and feature extractor
model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

# Load the dataset
dataset = load_dataset(root_folder, "audio", split="train")

# Function to extract features from an audio file
def extract_features(audio_path):
    # Load the audio file
    audio_array = load_audio(audio_path)

    # Extract features using the feature extractor
    inputs = feature_extractor(audio_array, return_tensors="pt")
    input_features = inputs.input_features
    decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id
    last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state

    return last_hidden_state

# Function to recursively traverse through directories and extract features
def process_files(directory):
    for root, dirs, files in os.walk(directory):
        for audio_file in files:
            # Extract features from the audio file
            audio_path = os.path.join(root, audio_file)
            features = extract_features(audio_path)

            # Remove unnecessary dimensions
            features = features.squeeze()

            # Save the features to a file
            features_file = os.path.join(features_folder, audio_file.replace(".wav", ".pt"))
            torch.save(features, features_file)

# Process files in the root folder and its subfolders
process_files(root_folder)


In [ ]:
import os
import torch
from transformers import AutoFeatureExtractor, WhisperModel
import soundfile as sf

# Set the directory path where your audio files are stored
directory_path = "/mnt/c/users/admin/desktop/F21RP/datasets/ravdess/audios"

model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

for root, dirs, files in os.walk(directory_path):
    for file in files:
        # Check if the file is an audio file
        if file.endswith(".wav"):
            # Retrieve the full path of the audio file
            audio_path = os.path.join(root, file)

            # Load the audio file
            audio_data, _ = sf.read(audio_path)

            # Convert the audio data into input features
            inputs = feature_extractor(audio_data, return_tensors="pt")
            input_features = inputs.input_features

            decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id
            last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state
            print(f"Last hidden state shape for {audio_path}: {list(last_hidden_state.shape)}")


In [ ]:
!nvidia-smi

In [ ]:
import os
import torch
from transformers import AutoFeatureExtractor, WhisperModel
import soundfile as sf



In [ ]:
# Set the directory path where your audio files are stored
directory_path = "/mnt/c/users/admin/desktop/F21RP/datasets/ravdess/audios/Audio_Speech_Actors_01-24/Actor_01/03-01-01-01-01-01-01.wav"

In [ ]:
# Check if a GPU is available
if torch.cuda.is_available():
    # Set the device to GPU
    device = torch.device("cuda")
else:
    device = torch.device("cpu")



In [ ]:
# Load the model and feature extractor
model = WhisperModel.from_pretrained("openai/whisper-base").to(device)
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")


In [ ]:

for root, dirs, files in os.walk(directory_path):
    for file in files:
        # Check if the file is an audio file
        if file.endswith(".wav"):
            # Retrieve the full path of the audio file
            audio_path = os.path.join(root, file)

            # Load the audio file
            audio_data, _ = sf.read(audio_path)

            # Convert the audio data into input features
            inputs = feature_extractor(audio_data, return_tensors="pt")
            input_features = inputs.input_features.to(device)

            decoder_input_ids = torch.tensor([[1, 1]], device=device) * model.config.decoder_start_token_id
            last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state

            # Move the last hidden state to CPU if needed
            last_hidden_state = last_hidden_state.to("cpu")

            print(f"Last hidden state shape for {audio_path}: {list(last_hidden_state.shape)}")


In [ ]:
print(device)

In [ ]:
import os
import torch
from transformers import AutoFeatureExtractor, WhisperModel
import librosa

# Set the directory path where your audio files are stored
directory_path = "/mnt/c/users/admin/desktop/F21RP/datasets/ravdess/audios"

model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

for root, dirs, files in os.walk(directory_path):
    for file in files:
        # Check if the file is an audio file
        if file.endswith(".wav"):
            # Retrieve the full path of the audio file
            audio_path = os.path.join(root, file)

            # Load the audio file using librosa
            audio_data, _ = librosa.load(audio_path, sr=None)

            # Convert the audio data into input features
            inputs = feature_extractor(audio_data, return_tensors="pt")
            input_features = inputs.input_features

            decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id
            last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state

            print(f"Last hidden state shape for {audio_path}: {list(last_hidden_state.shape)}")


In [ ]:
import os
import torch
from transformers import AutoFeatureExtractor, WhisperModel
import librosa
import numpy as np

# Set the directory path where your audio files are stored
directory_path = "/mnt/c/users/admin/desktop/rp/dataset/Ravdess_Audio_Visual"

# Set the output directory path to save the feature file
output_directory = "/mnt/c/users/admin/desktop/rp/dataset/features/audio"

model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

features = []  # List to store the extracted features

for root, dirs, files in os.walk(directory_path):
    for file in files:
        # Check if the file is an audio file
        if file.endswith(".wav"):
            # Retrieve the full path of the audio file
            audio_path = os.path.join(root, file)

            # Load the audio file using librosa
            audio_data, _ = librosa.load(audio_path, sr=None)

            # Convert the audio data into input features
            inputs = feature_extractor(audio_data, return_tensors="pt")
            input_features = inputs.input_features

            # Create dummy decoder input IDs
            decoder_input_ids = torch.tensor([[1]])  # Provide a dummy value

            # Perform model inference
            last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state

            # Append the extracted feature to the list
            features.append(last_hidden_state.detach().numpy())

# Concatenate all the extracted features
concatenated_features = np.concatenate(features)

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Save the concatenated features as a single file
output_file_path = os.path.join(output_directory, "features.npy")
np.save(output_file_path, concatenated_features)

print("Feature extraction completed. Features saved to:", output_file_path)


In [1]:
import os
import torch
from transformers import AutoFeatureExtractor, WhisperModel
import numpy as np
import torchvision.io as io

# Set the directory path where your video files are stored
directory_path = "/mnt/c/users/admin/desktop/rp/dataset/Ravdess_Audio_Visual"

# Set the output directory path to save the feature file
output_directory = "/mnt/c/users/admin/desktop/rp/dataset/features/audio"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = WhisperModel.from_pretrained("openai/whisper-base").to(device)
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

features = []  # List to store the extracted features

for root, dirs, files in os.walk(directory_path):
    for file in files:
        # Check if the file is a video file
        if file.endswith(".mp4"):
            # Retrieve the full path of the video file
            video_path = os.path.join(root, file)

            # Load the video file using torchvision.io
            video, audio, info = io.read_video(video_path, pts_unit="sec")
            audio_data = audio.squeeze(dim=0).numpy()

            # Convert the audio data into input features
            inputs = feature_extractor(audio_data, return_tensors="pt")
            input_features = inputs.input_features.to(device)

            # Create dummy decoder input IDs
            decoder_input_ids = torch.tensor([[1]]).to(device)  # Provide a dummy value

            # Perform model inference
            last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state

            # Append the extracted feature to the list
            features.append(last_hidden_state.detach().cpu().numpy())

# Concatenate all the extracted features
if len(features) > 0:
    concatenated_features = np.concatenate(features)

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Save the concatenated features as a single file
    output_file_path = os.path.join(output_directory, "features.npy")
    np.save(output_file_path, concatenated_features)

    print("Feature extraction completed. Features saved to:", output_file_path)
else:
    print("No valid audio files found in the directory:", directory_path)


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
2023-06-21 22:38:17.201351: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 22:38:17.875970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/epospiky/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

...

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

Feature extraction completed. Features saved to: /mnt/c/users/admin/desktop/rp/dataset/features/audio/features.npy
